# OBTENCION DE DATOS
**IMPORTAR LIBRERIAS Y OBTENER EL DATASET**

En primer lugar, importaremos las siguientes librerias necesarias para el analisis y procederemos a leer y mostrar el dataset.

In [1]:
import re
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.0f}'.format
warnings.filterwarnings('ignore')

In [2]:
url ='https://raw.githubusercontent.com/simonzanetti/2023.2-SysArmy-IT-Salaries-Survey/main/dataset.csv'
df = pd.read_csv(url)

#display(df.head(1))
#print("Shape: " + str(df.shape))

# LIMPIEZA DEL DATASET
**ORDENAR COLUMNAS**

El primer paso de nuestra limpieza sera traducir al ingles el nombre de las columnas y eliminar aquellas que no sean relevantes para el analisis. Tambien ordene las columnas en las categorias:
- Empleado
- Trabajo
- Empresa
- Sueldo
- Herramientas
- Estudios

Las columnas eliminadas fueron:
- 'work_country': La totalidad de los encuestados es de Argentina
- 'ARS/USD_exchange': No solo que el porcentaje de los que cobra parte o la totalidad de su sueldo en dolares es pequeño, sino tambien que existen varios tipos de cotizacion y no se aclara cual se usa en cada caso, ademas que la cotizacion suele variar segun las semanas, horas o dias, por lo que es una variable inestable y he decidido eliminarla.
- 'is:number': Una variable que no vamos a utilizar.

In [74]:
columns_names = ['work_country','work_province','work_type','work_contract_type',
                'last_month_gross_salary','last_month_net_salary','salary_in_usd',
                'ARS/USD_exchange','salary_has_bonus','salary_bonus_tied_to',
                '2023_salary_adjusment_times','percentage_adjustment',
                'last_adjustment_month','last_semester_salary_comparison',
                'work_place_benefits','salary_satisfaction','work_title','years_experience',
                'years_in_company','years_in_position','people_in_charge','platforms',
                'programming_languages','frameworks','databases','qa_testing_tools',
                'company_size','work_mode','office_days_number(hybrid)','work_place_satisfaction',
                'AI_tools_use','finish_survey(1)','highest_level_studies','status','degree/specialization',
                'university/school','finish_survey(2)','work_on-call_duty','salary_on-call_duty',
                'is_number','finish_survey(3)','age','gender']

for i, nuevo_nombre in enumerate(columns_names):
    df.rename(columns={df.columns[i]: nuevo_nombre}, inplace=True)

salaries = pd.concat([df.iloc[:,0:2],df.iloc[:,41:],df.iloc[:,16],
                      df.iloc[:,2:4],df.iloc[:,27],df.iloc[:,17:21],df.iloc[:,26:27],df.iloc[:,28:30],df.iloc[:,14],
                      df.iloc[:,4:14],df.iloc[:,15],df.iloc[:,21:26],df.iloc[:,30:41]],
                      axis=1)

salaries.drop(inplace=True, columns=[
                                    'work_country','ARS/USD_exchange','2023_salary_adjusment_times',
                                    'percentage_adjustment','last_adjustment_month',
                                    'last_semester_salary_comparison','finish_survey(1)','finish_survey(2)',
                                    'work_on-call_duty','salary_on-call_duty','is_number','finish_survey(3)'
                                    ])

print(salaries.dtypes)

work_province                  object
age                             int64
gender                         object
work_title                     object
work_type                      object
work_contract_type             object
work_mode                      object
years_experience              float64
years_in_company              float64
years_in_position             float64
people_in_charge                int64
company_size                   object
office_days_number(hybrid)      int64
work_place_satisfaction         int64
work_place_benefits            object
last_month_gross_salary       float64
last_month_net_salary         float64
salary_in_usd                  object
salary_has_bonus               object
salary_bonus_tied_to           object
salary_satisfaction             int64
platforms                      object
programming_languages          object
frameworks                     object
databases                      object
qa_testing_tools               object
AI_tools_use

**LIMPIEZA Y TRANSFORMACION DE NANS**

In [6]:
print('Cantidad de Nans por limpiar: ')
print(salaries.isna().sum()[salaries.isna().sum() > 0])

#SALARY_IN_USD
#print(salaries['salary_in_usd'].unique())
salaries['salary_in_usd'].fillna('No cobro mi salario en dolares',inplace=True)

#TOOLS
salaries.loc[:, 'platforms':'AI_tools_use'] = salaries.loc[:, 'platforms':'AI_tools_use'].fillna('Ninguna de las anteriores')

#STUDIES
#print(df['finish_survey(1)'].unique())
#print(df['highest_level_studies'].unique())
for columns in ('highest_level_studies','status','degree/specialization','university/school'):
    salaries.loc[(df['finish_survey(1)'] == 'Terminar encuesta') |
                 (df['finish_survey(1)'] == 'Responder sobre guardias'), columns] = 'Prefiero no responder'

for columns in ('degree/specialization','university/school'):
    salaries.loc[(df['highest_level_studies'] == 'Secundario'), columns] = 'Prefiero no responder'

Cantidad de Nans por limpiar: 
last_month_net_salary    239
degree/specialization     15
university/school        350
dtype: int64


**LIMPIEZA DE DUPLICADOS**


**NORMALIZACION DE VALORES**

En algunas columnas de este dataset nos encontraremos con una gran dispersion de opciones, con valores incorrectos o con valores que no representan de forma adecuada la respuesta del encuestado. Debido a ello necesitamos realizar una normalizacion de valores, en algunos casos en formato string y en otros casos en formato numerico.

Con la necesidad de reducir el tamaño del codigo, he creado una funcion que toma un dataset, una columna y un diccionario que contiene un mapeo regex, devolviendo un nuevo dataset normalizado.

Mediante el metodo pd.nunique() podemos ver la cantidad de opciones por cada variable, notando ciertas irregularidades:
- **'gender'**: Presenta 144 opciones diferentes de generos, lo cual nos muestra que exste una dispersion erronea en los datos.
- **'work title'**: Presenta una gran dispersion de valores.
- **'people_in_charge'**: Hay 58 opciones que varian entre 0 y 300. Lo ideal seria agruparlas en categorias mas generales, pero para ello se necesita realizar un analisis para visualizar outliers y correlacion con otras variables para detectar valores falsos.  
- **'company_size'**: Hay 11 categorias que varian entre 0 y 10000. Lo ideal seria agruparlas en categorias mas generales, pero para ello se necesita realizar un analisis para visualizar outliers y correlacion con otras variables para detectar valores falsos.
- **'office_days_number(hybrid)'**: Las opciones '0' y '5' no son correctas, ya que representan a los empleados que o no van nunca a la oficina o que van todos los dias, lo cual no condice con el modo de trabajo hibrido. Se necesita un analisis previo antes de normalizar.



**FUNCIONES AUXILIARES NORMALIZACION**

In [7]:
def normalize_dataframe(df_in, column_name, regex_assign_map):
    df_out = df_in.copy()
    for regex, value in regex_assign_map.items():
        print("\nElementos de " + str.capitalize(column_name) + " que integran la categoria " + regex +':')
        compiled_regex = re.compile(regex, re.I)
        mask = df_in[column_name].str.strip().str.match(compiled_regex, na=False)
        print(df_in[df_out[column_name].str.strip().str.match(compiled_regex, na=False)][column_name].unique())
        df_out.loc[mask, column_name] = value
    return df_out

def extract_values(column):
    value_counts_series = (column.str.extractall(r"(?P<count>[^,]+),? ?")["count"].value_counts(sort=True))
    df_out = pd.DataFrame({'item': value_counts_series.index, 'counts': value_counts_series.values})
    return df_out

In [25]:
#print("Cantidad de valores que toma cada variable: ")
#print(salaries.nunique())

**WORKER**

In [ ]:
#GENERO
#print(salaries['gender'].unique())

salaries_temp = normalize_dataframe(salaries, 'gender',
    {
        "^(?!.*(mujer)).*hombre.*|.*macho.*|.*muchacho.*|.*varoo.*|.*masculino.*|^(?!.*(?:trans|bi|\b(var[oó]n cis)\b))var[oó]n" : "Varón Cis",
        "^(?!.*(?:trans|bi|ag[eé]nero|var[oó]n|hombre|hombres)).*mujer|.*mujer de cuando solo existian hombres y mujeres*." : "Mujer Cis",
        "^(?=.*mujer|var[oó]n)(?=.*trans)(?!.*nb).*" : "Otros",
        "^(?=.*mujer|var[oó]n)\b(bi)\b" : "Otros",
        ".*ag[eé]nero.*" : "Otros",
        ".*nb.*|.*no binario.*|.*no binarie.*|big[eé]nero|.*varon bi.*" : "Otros",
        ".*fluido.*" : "Otros",
        ".*queer.*" : "Otros",
        ".*puto.*|.*marika.*|.*lesbiana.*" : "Otros",
    })
    
print('\nElementos de Gender que no entran en las categorias anteriores')
print(salaries_temp['gender'][~salaries_temp['gender'].isin(['Varón Cis','Mujer Cis','Prefiero no decir','Otros'])].unique())
salaries_temp['gender'] = np.where(salaries_temp['gender'].isin(['Varón Cis','Mujer Cis','Prefiero no decir','Otros']), salaries_temp['gender'], 'No especificados o erroneos')

**WORK**

In [ ]:
#WORK_TITLE
#print(salaries['work_title'].unique())

salaries1 = normalize_dataframe(salaries, 'work_title',
    {
        "bi|data analyst" : "BI Analyst / Data Analyst",
        "manager" : "BI Analyst / Data Analyst",
        "analista" : "BI Analyst / Data Analyst",
    })

fig = plt.figure(figsize=(15, 5))
ax = sns.countplot(y=salaries['work_title'], color='steelblue', order=df["work_title"].value_counts().index)
plt.title("Distribucion de encuestados por posición",loc='center')
plt.xlabel("Cantidad")
plt.ylabel('')
#ax.bar_label(ax.containers[0], padding=3)
ax.margins(y=0.015)

In [9]:
# WORK_CONTRACT TYPE
#print(salaries["work_contract_type"].unique())
salaries_temp['work_contract_type'] = salaries['work_contract_type'].replace({
                                                                                'Staff (planta permanente)' : 'Planta permanente',
                                                                                'Tercerizado (trabajo a través de consultora o agencia)' : 'Tercerizado',
                                                                                'Contractor' : 'Contratista',
                                                                                'Participación societaria en una cooperativa': 'Participación societaria'
                                                                            })

In [10]:
# WORK_MODE
#print(salaries["work_mode"].unique())
salaries_temp['work_mode'] = salaries['work_mode'].replace({
                                                            '100% presencial' : 'Presencial',
                                                            'Híbrido (presencial y remoto)' : 'Híbrido',
                                                            '100% remoto' : 'Remoto'
                                                          })


**COMPANY**

In [84]:
#PEOPLE_IN_CHARGE
#print(sorted(salaries['people_in_charge'].unique()))

salaries_temp['people_in_charge'] = salaries['people_in_charge'].mask(salaries['people_in_charge'] > 5, '+ 5')

In [82]:
#COMPANY_SIZE
#print(salaries['company_size'].unique())
salaries_temp['company_size'] = salaries['company_size'].replace(
                                {
                                'De 2 a 10 personas' : 'Microempresa (2 a 10 trabajadores)',
                                'De 11  a 50  personas' : 'Pequeña empresa (11 a 50 trabajadores)',
                                'De 51 a 100 personas' : 'Mediana empresa (50 a 200 trabajadores)',
                                'De 101 a 200 personas' : 'Mediana empresa (50 a 200 trabajadores)',
                                'De 201 a 500 personas'  : 'Empresa grande (+ 200 trabajadores)', 
                                'De 501 a 1000 personas' : 'Empresa grande (+ 200 trabajadores)',
                                'De 1001 a 2000 personas' : 'Empresa grande (+ 200 trabajadores)',
                                'De 2001a 5000 personas' : 'Empresa grande (+ 200 trabajadores)', 
                                'De 5001 a 10000 personas' : 'Empresa grande (+ 200 trabajadores)',
                                'Más de 10000 personas' : 'Empresa grande (+ 200 trabajadores)'
                                })

In [ ]:
#OFFICE_DAYS_NUMBER
'''
ESPERAR ANALISIS
print(sorted(salaries['office_days_number(hybrid)'].unique()))
#sns.countplot(y=salaries['office_days_number(hybrid)'], color='steelblue',order=salaries['office_days_number(hybrid)'].value_counts().index)
sns.heatmap(pd.crosstab(salaries['office_days_number(hybrid)'], salaries['work_mode']), annot=True, fmt='d', cmap='YlGnBu')
'''

In [ ]:
#WORK_PLACE_SATISFACTION
'''
ES NECESARIO ALGUN TIPO DE CLUSTERING PARA AGRUPAR
#WORK_PLACE_SATISFACTION
#print('work_place_satisfaction'].unique())
salaries['work_place_satisfaction'] = salaries['work_place_satisfaction'].replace({
                                                            1:'Muy insatisfecho',2:'Muy insatisfecho',
                                                            3:'Escasamente satisfecho',4:'Escasamente satisfecho',5: 'Escasamente satisfecho',
                                                            6:'Ampliamente satisfecho',7:'Ampliamente satisfecho',8: 'Ampliamente satisfecho',
                                                            9:'Muy satisfecho',10:'Muy satisfecho'
                                                            })
'''

In [31]:
#WORK_BENEFITS
#print(salaries['work_place_benefits'].unique())
benefits_count = extract_values(salaries['work_place_benefits'])

**SALARY**

In [33]:
#SALARY_SATISFACTION
#print(salaries['salary_satisfaction'].unique())
salaries_temp['salary_satisfaction'] = salaries['salary_satisfaction'].replace({
                                                                    1:'Muy insatisfecho',2:'Escasamente satisfecho',
                                                                    3:'Ampliamente satisfecho',4:'Muy satisfecho'
                                                                               })

**TOOLS**

In [34]:
#PLATFORMS
platforms_count = extract_values(salaries['platforms'])

#PROGRAMMING_LENGUAGES
programming_languages_count = extract_values(salaries['programming_languages'])

#FRAMEWORKS
frameworks_count = extract_values(salaries['frameworks'])

#DATABASES
databases_count = extract_values(salaries['databases'])

#QA_TESTING_TOOLS
qa_testing_tools_count = extract_values(salaries['qa_testing_tools'])

#AI_TOOLS_USE
#print(salaries['AI_tools_use'].unique())
salaries_temp['AI_tools_use'] = salaries['AI_tools_use'].replace({
                                                            0:'No utilizadas',1:'Escasamente utlizadas',
                                                            2:'Escasamente utlizadas',3:'Ampliamente utilizadas',
                                                            4:'Ampliamente utilizadas',5:'Muy utilizadas'
                                                                })


**STUDIES**

In [ ]:
#DEGREE/SPECIALIZATION
#print(salaries['degree/specialization'].unique())

salaries_temp = normalize_dataframe(salaries, 'degree/specialization',
    {
        ".*ingenier[ií]a.* .*sistemas.*" : "Ingenieria en Sistemas",
        #".*ana.* .*sistema.* " : "Analista de Sistemas",
        ".*empresa.* " : "Administracion de Empresas",
        ".*economia.* " : "Lic. en Economia",
        ".*desarrollo.* .*web.*" : "Tec. en Desarrollo Web",
        ".*desarrollo.* .*movil.*" : "Tec. en Desarrollo de Aplicaciones Moviles",
        ".*desarrollo.* .*software.*" : "Tec. en Desarrollo de Software",
        ".*juego.* .*game.*" : "Tec. en Desarrollo de Videojuegos",
        "^(?!.*(?:higiene|lic|master|esp)).*seguridad.*" : "Tec. en Ciberseguridad/ Seguridad Informatica",
        "^(?!.*(?:higiene|tec|master|esp)).*seguridad.*" : "Lic. en Ciberseguridad/ Seguridad Informatica",
        ".*tec.* .*redes.*" : "Tec. en Redes Informaticas y Telecomunicaciones",
        ".*lic.* .*redes.*" : "Lic. en Redes Informaticas y Telecomunicaciones",
        "^(?!.*(?:sistema))(?=.*lic.* .*computaci[oó]n.*)" : "Lic. en Ciencia de la Computación",
        ".*ing.* .*computaci[oó]n.*" : "Ingeniería en Computación",
        ".*ana.* .*computaci[oó]n.*" : "Analista en Computación",
        ".*computaci[oó]n.*" : "Ingeniería en Computación",
        #".*dat.* " : "Tec. en Desarrollo de Software",
        #".*lic.* " : "Tec. en Desarrollo de Software",
        #"^(?!.*(?:robot|admin))(?=.*lic.*sistemas)" : "Licenciatura en Sistemas",
    })


In [45]:
salaries = salaries_temp
salaries.to_csv('cleaned_dataset.csv', index=False)
print(salaries['company_size'])

0              201 a 500
1              201 a 500
2               11  a 50
3             501 a 1000
4               51 a 100
5               11  a 50
6                 2 a 10
7              201 a 500
8              201 a 500
9              101 a 200
10              51 a 100
11            501 a 1000
12              11  a 50
13             101 a 200
14           1001 a 2000
15            501 a 1000
16             201 a 500
17              51 a 100
18      1 (solamente yo)
19              51 a 100
20             101 a 200
21             201 a 500
22          Más de 10000
23             101 a 200
24              11  a 50
25            501 a 1000
26            501 a 1000
27                2 a 10
28             201 a 500
29              11  a 50
30             101 a 200
31             101 a 200
32              11  a 50
33            501 a 1000
34              11  a 50
35          Más de 10000
36           1001 a 2000
37              11  a 50
38              11  a 50
39          Más de 10000
